In [4]:
from cornac.data import Reader
from cornac.datasets import movielens
from cornac.data import Dataset, FeatureModality
from cornac.eval_methods import RatioSplit, StratifiedSplit
from cornac.metrics import RMSE
from cornac.models import  ItemKNN, UserKNN, NMF, BPR, MF,VAECF
# from MF import MF
import pandas as pd
import numpy as np
import cornac
import math
# import seaborn as sns
# import matplotlib.pyplot as plt

In [5]:
rating_data_pd = pd.read_csv(
    "../data/ml-100K/indexed_interactions.csv",
    sep="\t",
    header=0,
    names=["userID", "itemID", "Rating", "Timestamp"],
)
rating_data = rating_data_pd.to_numpy()
rating_data.__len__()
rating_data_pd

,userID,itemID,Rating,Timestamp
0,0,0,3,881250949
1,1,1,3,891717742
2,2,2,1,878887116
3,3,3,2,880606923
4,4,4,1,886397596
...,...,...,...,...
99273,875,173,3,880175444
99274,708,247,5,879795543
99275,37,981,1,874795795
99276,58,442,2,882399156


In [6]:

movie = '../data/ml-100K/i_id_mapping_genre.csv'

df_movie = pd.read_csv(movie, sep='\t')
print(f'shape: {df_movie.shape}')


df_movie.columns = ["item_id", "Name","genres","itemID"]
df_movie[:4]
movies = df_movie.sort_values(by="itemID")
unique_genres = [
    "Action",
    "Thriller",
    "Romance",
    "Western",
    "Children's",
    "Mystery",
    "Fantasy",
    "Film-Noir",
    "Documentary",
    "Comedy",
    "Adventure",
    "Sci-Fi",
    "Horror",
    "Crime",
    "Musical",
    "War",
    "Animation",
    "Drama",
]
for genre in unique_genres:
    movies[genre] = 0
    
for index, row in movies.iterrows():
    genres = row["genres"].split("|")
    for genre in genres:
        movies.at[index, genre] = 1
movies


movies

shape: (1348, 4)


,item_id,Name,genres,itemID,Action,Thriller,Romance,Western,Children's,Mystery,...,Documentary,Comedy,Adventure,Sci-Fi,Horror,Crime,Musical,War,Animation,Drama
0,242,Kolya (1996),Comedy,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,302,L.A. Confidential (1997),Thriller|Mystery|Film-Noir|Crime,1,0,1,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
2,377,Heavyweights (1994),Children's|Comedy,2,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
3,51,Legends of the Fall (1994),Romance|Western|War|Drama,3,0,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,1
4,346,Jackie Brown (1997),Crime|Drama,4,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1343,1335,American Buffalo (1996),Drama,1343,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1344,1251,A Chef in Love (1996),Comedy,1344,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1345,1233,Nénette et Boni (1996),Drama,1345,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1346,1380,Hollow Reed (1996),Drama,1346,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [7]:
movies = movies.drop(columns=["item_id"])
movies = movies.sort_values(by="itemID")
movies

,Name,genres,itemID,Action,Thriller,Romance,Western,Children's,Mystery,Fantasy,...,Documentary,Comedy,Adventure,Sci-Fi,Horror,Crime,Musical,War,Animation,Drama
0,Kolya (1996),Comedy,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,L.A. Confidential (1997),Thriller|Mystery|Film-Noir|Crime,1,0,1,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
2,Heavyweights (1994),Children's|Comedy,2,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
3,Legends of the Fall (1994),Romance|Western|War|Drama,3,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,1
4,Jackie Brown (1997),Crime|Drama,4,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1343,American Buffalo (1996),Drama,1343,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1344,A Chef in Love (1996),Comedy,1344,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1345,Nénette et Boni (1996),Drama,1345,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1346,Hollow Reed (1996),Drama,1346,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [8]:
users = pd.read_csv("../data/ml-100k/u_id_mapping_demographic_.csv", sep="\t")
# users


users = users.sort_values(by="userID")

users = users.drop(columns=users.columns[0])
gender_map = {"M": 0, "F": 1}
users["Gender"] = users["Gender"].map(gender_map)
# user_features_numpy = users.to_numpy()
users

,Age,Gender,Occupation,ZipCode,userID,Occupation_Code
195,49,0,writer,55105,0,20
185,39,1,executive,00000,1,6
21,25,0,writer,40206,2,20
243,28,0,technician,80525,3,19
165,47,0,educator,55113,4,3
...,...,...,...,...,...,...
938,26,1,student,33319,938,18
935,24,0,other,32789,939,13
929,28,1,scientist,07310,940,17
919,30,1,artist,90008,941,1


In [10]:
dataset = rating_data
dataset

array([[        0,         0,         3, 881250949],
       [        1,         1,         3, 891717742],
       [        2,         2,         1, 878887116],
       ...,
       [       37,       981,         1, 874795795],
       [       58,       442,         2, 882399156],
       [      304,       504,         3, 879959583]])

In [71]:

ratio_split = StratifiedSplit(
    data=dataset, test_size=0.2, rating_threshold=0.0, seed=123, verbose=True, chrono=True
)

hr_10 = cornac.metrics.HitRatio(k=20)
ndcg_10 = cornac.metrics.NDCG(k=10)


model = MF(
    k=20, max_iter=20, learning_rate=0.001, seed=123, name="mfe20", batch_size=256
)

models = [model]
cornac.Experiment(
    ratio_split, models=models, metrics=[hr_10, ndcg_10]
).run()

rating_threshold = 0.0
exclude_unknowns = True
---
Training data:
Number of users = 943
Number of items = 1348
Number of ratings = 79039
Max rating = 5.0
Min rating = 1.0
Global mean = 3.6
---
Test data:
Number of users = 943
Number of items = 1348
Number of ratings = 20239
Number of unknown users = 0
Number of unknown items = 0
---
Total users = 943
Total items = 1348

[mfe20] Training started!

[mfe20] Evaluation started!


Ranking: 100%|██████████| 943/943 [00:00<00:00, 6758.84it/s]


TEST:
...
      | HitRatio@20 | NDCG@10 | Train (s) | Test (s)
----- + ----------- + ------- + --------- + --------
mfe20 |      0.4751 |  0.0620 |    0.0539 |   0.1503



In [72]:
user_ids = users["userID"].to_numpy()
item_ids = movies["itemID"].to_numpy()
user_ids.__len__()

943

In [83]:
# get the top_k ratings for all users:
top_k = 20
reco_matrix = np.zeros((len(models), len(user_ids), top_k), dtype=int)
reco_matrix_mapped_items = np.zeros(
    (len(models), len(user_ids), len(item_ids)), dtype=int
)
reco_matrix_mapped_scores = np.zeros(
    (len(models), len(user_ids), len(item_ids)), dtype=float
)
reco_matrix_all = np.zeros((len(models), len(user_ids), len(item_ids)), dtype=int)


for u in user_ids:
    for i in range(len(models)):
        reco_items = models[i].recommend(u)
        items_mapped, mapped_scores = models[i].rank(
            user_idx=u, item_indices=list(item_ids)
        )
        reco_matrix_mapped_items[i][u] = items_mapped
        reco_matrix_mapped_scores[i][u] = mapped_scores
        reco_matrix_all[i][u] = reco_items
        reco_matrix[i][u] = reco_items[:top_k]

        # print(reco_matrix[0][3])

In [76]:
test_set_data = pd.DataFrame(ratio_split.test_set.uir_tuple).transpose()
test_set_data.columns = ["uid", "iid", "rating"]
test_set_data = test_set_data.astype({"uid": "int", "iid": "int", "rating": "int"})
r_global_uid_map = {v: k for k, v in ratio_split.global_uid_map.items()}
r_global_iid_map = {v: k for k, v in ratio_split.global_iid_map.items()}

test_set_data["uid"] = test_set_data["uid"].map(r_global_uid_map)
test_set_data["iid"] = test_set_data["iid"].map(r_global_iid_map)

In [78]:
train_set_data = pd.DataFrame(ratio_split.train_set.uir_tuple).transpose()
train_set_data.columns = ["uid", "iid", "rating"]
train_set_data = train_set_data.astype({"uid": "int", "iid": "int", "rating": "int"})
r_global_uid_map = {v: k for k, v in ratio_split.global_uid_map.items()}
r_global_iid_map = {v: k for k, v in ratio_split.global_iid_map.items()}

train_set_data["uid"] = train_set_data["uid"].map(r_global_uid_map)
train_set_data["iid"] = train_set_data["iid"].map(r_global_iid_map)

In [70]:
test_set_data.to_csv("testing_set_seed123_ml100k.csv", index=False, header=False)


In [81]:
train_set_data.to_csv("training_set_seed123_ml100k.csv", index=False, header=False)


In [ ]:
np.save("reco_matrix_mf_100k",reco_matrix)
